Azure GET CLI Details

In [2]:
import os
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.mgmt.compute import ComputeManagementClient
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
print("Subscription ID:", subscription_id)
credential = DefaultAzureCredential()
print("Using DefaultAzureCredential -> will authenticate via env vars or CLI login")
compute_client = ComputeManagementClient(credential, subscription_id)
print(compute_client)
print("Compute client ready for region-by-region VM SKU queries")

Subscription ID: 00000000-0000-0000-0000-000000000000
Using DefaultAzureCredential -> will authenticate via env vars or CLI login
Compute client ready for region-by-region VM SKU queries


In [ ]:
def collect_all_vm_sizes(compute_client: ComputeManagementClient):

    rows = []
    for sku in compute_client.resource_skus.list():
        # Only virtual machine SKUs
        if sku.resource_type.lower() != 'virtualmachines':
            continue

        # Map capabilities list to a dict for lookup
        caps = {c.name: c.value for c in sku.capabilities}

        # Expand each location the SKU is offered in
        for loc_info in sku.location_info:
            loc = loc_info.location
            rows.append({
                'Region':              loc,
                'VMSize':              sku.name,
                'vCPUs':               int(caps.get('vCPUs', 0)),
                'Memory (GiB)':        float(caps.get('MemoryGB', 0)),
                'OS Disk Size (GB)':   float(caps.get('OSVhdSizeMB', 0)) / 1024,
                'Temp Disk Size (GB)': float(caps.get('ResourceDiskSizeMB', 0)) / 1024,
                'Max Data Disks':      int(caps.get('MaxDataDiskCount', 0)),
                'Network BW (Gbps)':   float(caps.get('NetworkBandwidthInGbps', 0)),
                'GPU Count':           int(caps.get('GPUs', 0)),
            })

    return pd.DataFrame(rows)

In [ ]:
df_vm_sizes = collect_all_vm_sizes(compute_client)

# 4. Inspect
print(df_vm_sizes.head())
print("Total SKUs:", len(df_vm_sizes))